In [1]:
# https://media.daum.net/ 에서 HTML 받아와서 주요 이슈 확인
from urllib.request import urlopen
from bs4 import BeautifulSoup

# HTTP 요청
req = urlopen("https://media.daum.net/")
# HTTP 응답 코드 확인 getcode()
print("HTTP Response Code:", req.getcode()) # 200번이면 정상
# 만약 응답 코드가 200이면 html을 받아오기
if req.getcode()==200:
    html = req.read()
    # encoding 변환
    html = html.decode("utf-8")
    print(html)
else:
    print("HTTP ERROR")

HTTP Response Code: 200

<!doctype html>
<html lang="ko">
<head>
	<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=Edge">



<meta property="og:author" content="Daum 뉴스" />
<meta property="og:site_name" content="다음뉴스" />
<meta property="og:title" content="홈"/>
<meta property="og:image" content="//t1.daumcdn.net/media/img-media/mobile/meta/news.png" />
<meta property="og:description" content="다음뉴스" />
<meta property="og:url" content="http://media.daum.net/"/>
<meta name="referrer" content="always" />

<meta name="svcdomain" content="media.daum.net" />

<title>홈 | 다음뉴스</title>


<link rel="shortcut icon" href="//m2.daumcdn.net/img-media/2010ci/Daum_favicon.ico">

<link rel="stylesheet" type="text/css" href="//t1.daumcdn.net/media/static/media-1276/dist/news/css/common.min.css">
<link rel="stylesheet" type="text/css" href="//t1.daumcdn.net/media/static/media-1276/dist/news/css/news.min.css">

<!--[if lte IE 7]>
<script type="text/javascript" src="//t1.daumcdn.net/medi

In [2]:
# 분석을 위해서 단순 html 텍스트를 -> DOM 트리 객체 형태로 변환
# BeautifulSoup
soup = BeautifulSoup(html, "html.parser") # html파서로 텍스트를 객체화
#print(soup)
# 페이지 정보 확인
print("페이지의 타이틀", soup.title.text)

# 주요 이슈 SELECTOR : ul.list_issue
list_issue = soup.select("ul.list_issue")
#print(list_issue, type(list_issue))
titles = list_issue[0].select("a")
print("count if titles:", len(titles))

from bs4 import Tag
for title in titles:
    #print(type(title)) # a 태그(span 태그들을 포함하고 있음 -> 제거, only title값만 가져오기 위함)
    for child in title.children:
        #print(child, type(child)) # 이중에서 tag만 걸러내야함 
        if isinstance(child, Tag):
            child.decompose() # 부모 태그에서 분리 
    print("이슈 타이틀:", title.text.strip())

페이지의 타이틀 홈 | 다음뉴스
count if titles: 5
이슈 타이틀: 선거제 개편 추진
이슈 타이틀: 국방과학연구소 폭발사고
이슈 타이틀: 지소미아 종료
이슈 타이틀: 황교안 대표 단식
이슈 타이틀: 한미 방위비 협상


In [3]:
# 대상 페이지 : https://movie.naver.com/movie/running/current.nhn
# 최신 영화 리스트 추출 

# html text 받아오기
req = urlopen("https://movie.naver.com/movie/running/current.nhn")
html = req.read().decode("utf-8")
# 페이지의 일부 확인
#print(html.strip()[:100])

soup = BeautifulSoup(html, "html.parser")
# 페이지의 정보 확인
print("페이지의 타이틀:", soup.title)
print("page title name:", soup.title.name) # 태그 이름 
print("page title string:", soup.title.string)
print("특정 태그의 부모:", type(soup.title.parent)) # 해당 타이틀 태그의 부모 확인 

# 페이지 소스를 보기 좋게 출력하기 : prettify()
print("네이버 영화:")
print(soup.prettify()[:200])

페이지의 타이틀: <title>현재 상영영화 : 네이버 영화</title>
page title name: title
page title string: 현재 상영영화 : 네이버 영화
특정 태그의 부모: <class 'bs4.element.Tag'>
네이버 영화:
<!DOCTYPE html>
<html lang="ko">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="no" http-equiv="imagetoolbar"/>
  <title>
   현재 상영영화 : 네이버 


In [4]:
# 분석과 추출 : 영화 랭킹 
currents = soup.select("ul.lst_detail_t1")[0]
titles = currents.select("dt", attrs={"class":"tit"})
movies = [] # 영화 랭킹을 담을 dict 

for title in titles:
    for child in title.children:
        if child.name == "a":
            # a Tag 찾음
            #print(child.text)
            movie_title = child.text
            movie_href = child['href']
            #print("movie_title:{}, movie url:{}".format(movie_title, movie_href))
            
            movie = {
                "title": movie_title, 
                "url" : movie_href
                }
            movies.append(movie)
# 영화 랭킹 출력
print("현재 영화 랭킹:")
for index, movie in enumerate(movies):
    print("{}위 - {}".format(index+1, movie))

현재 영화 랭킹:
1위 - {'title': '겨울왕국 2', 'url': '/movie/bi/mi/basic.nhn?code=136873'}
2위 - {'title': '블랙머니', 'url': '/movie/bi/mi/basic.nhn?code=189053'}
3위 - {'title': '신의 한 수: 귀수편', 'url': '/movie/bi/mi/basic.nhn?code=179159'}
4위 - {'title': '82년생 김지영', 'url': '/movie/bi/mi/basic.nhn?code=179482'}
5위 - {'title': '터미네이터: 다크 페이트', 'url': '/movie/bi/mi/basic.nhn?code=167605'}
6위 - {'title': '엔젤 해즈 폴른', 'url': '/movie/bi/mi/basic.nhn?code=163831'}
7위 - {'title': '윤희에게', 'url': '/movie/bi/mi/basic.nhn?code=182387'}
8위 - {'title': '아이리시맨', 'url': '/movie/bi/mi/basic.nhn?code=154298'}
9위 - {'title': '대통령의 7시간', 'url': '/movie/bi/mi/basic.nhn?code=189071'}
10위 - {'title': '어제 일은 모두 괜찮아', 'url': '/movie/bi/mi/basic.nhn?code=173842'}
11위 - {'title': '날씨의 아이', 'url': '/movie/bi/mi/basic.nhn?code=181114'}
12위 - {'title': '좀비랜드: 더블 탭', 'url': '/movie/bi/mi/basic.nhn?code=74407'}
13위 - {'title': '프렌드 존', 'url': '/movie/bi/mi/basic.nhn?code=187548'}
14위 - {'title': '삽질', 'url': '/movie/bi/mi/basic.nhn?co

In [21]:
# 네이버 영화 페이지에서 포스터 이미지 크롤링 
#print(html.strip()[:100])
# 이미지의 변수는 모두 같기 때문에 alt 변수를 이용해 이미지를 가지고 온다, 이미지 타입 .jpg 로 적재해야함 

# 저장 디렉토리 : D:\crawl
import os
if not os.path.exists("/crawl"):
    os.mkdir("/crawl") # crawl파일이 없으면 새로 생성 
# html -> 객체 : BeautifulSoup
soup = BeautifulSoup(html, "html.parser")
movies = soup.select("ul.lst_detail_t1 > li")

# 다운로드를 위한 import : urletrieve
from urllib.request import urlretrieve
#print(movies)
# 개별 영화 Loop
for movie in movies:
    img = movie.find("img")
    #print("img:", img)
    img_name = img['src']
    img_alt = img['alt']
    #print(img_name, img_alt) # 전부 movie_image 이기 떄문에 확장자 .jpg만 뽑아내기
    if len(img_name.split("?")) == 1:
        pass
    else: # image_name에 ? 가 붙어있다 
        img_name = img_name.split("?")[0]
    #rint("이미지 파일명:", img_name)
    # 확장자 추출
    ext = img_name.split(".")[-1] # 가장 마지막 값
    #print("이미지 확장자:", ext)
    # img의 alt값을 파일명으로 사용하기 적합하도록 변경
    import re
    # 불용문자를 -로 교체 
    fname = re.sub(r"[\s:,?]", "-", img_alt.strip()) # 공백문자, :, ,, ? 이면 -로 교체하겠다
    # 최종적으로 저장용 파일명 생성
    full_name = "{}.{}".format(fname, ext)
    full_name = "/".join(["/crawl", full_name])
    #print("최종파일명:", full_name)
    
    # 다운로드 : urletrieve(이미지url, 실제 저장할 파일명)
    urlretrieve(img['src'], full_name)
print("=====이미지 저장 완료")

=====이미지 저장 완료
